En una red social sobre cine, tenemos un fichero ratings.txt compuesta por el código de la película, el código del usuario, la calificación asignada y el timestamp de la votación con el siguiente formato:


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968

Se pide crear dos script en Python, así como los comandos necesarios para ejecutarlos (mediante spark-submit) para:

Obtener para cada película, la nota media de todas sus votaciones.
Películas cuya nota media sea superior a 3.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() 
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [4]:
rdd = sc.textFile('../ratings.txt')
rdd.take(5)

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275',
 '1::2355::5::978824291']

In [5]:
# Obtener para cada película, la nota media de todas sus votaciones.
rddRatings = rdd.map(lambda x: x.split('::')).map(lambda x: (x[0], (int(x[2]), 1)))
rddRatings.count()

655982

In [6]:
rddSuma = rddRatings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
rddSuma.take(5)

/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('1', (222, 53)),
 ('4', (88, 21)),
 ('8', (540, 139)),
 ('9', (396, 106)),
 ('10', (1650, 401))]

In [9]:
rddMedia = rddSuma.map(lambda x: (x[0], x[1][0]/x[1][1]))
rddMedia.take(3)

[('1', 4.188679245283019), ('4', 4.190476190476191), ('8', 3.884892086330935)]

In [10]:
rddMediaMV = rddSuma.mapValues(lambda x: x[0]/x[1])
rddMediaMV.take(3)

[('1', 4.188679245283019), ('4', 4.190476190476191), ('8', 3.884892086330935)]